 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: ``.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [326]:
import os
import selenium
import token
from urllib import response
import responses
import requests    # tokeny autoryzacyjne
import pandas as pd    # do wczytania csv
import time    # do opóźnień między kolejnymi żądaniami do API
import json    # ta bibl. pomoże w ich przetwarzaniu danych API
import sqlalchemy    # do wgrywania zebranych danych do bazy danych
import logging    # do logowania błędów, np. przekroczenia limitu API
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import csv


In [327]:
output_directory = "C:\\AAAAAAAAAAAa\\CODERSLAB\\KURS_ANALITYK\\PORTFOLIO\\Pliki_do_Portfolio_Lab\\data\\Airport"

# pobieranie z csv
airports_ids = pd.read_csv(os.path.join(output_directory, "airports.csv"))["origin_airport_id"].tolist()
print(airports_ids)

url = "https://api-datalab.coderslab.com/v2/docs/#"


# konfiguracja chromedriver
options = webdriver.ChromeOptions()
prefs = {"download.default_directory": output_directory}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)
print("start processing")


# utworzenie list do rozdzielenia pustych i pełnych lotnisk
airport_data = []
failed_airports = []


def download_and_clear(driver):
    try:
        download_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.download-contents")))
        driver.execute_script("arguments[0].scrollIntoView(true);", download_button)
        driver.execute_script("arguments[0].click();", download_button)

        clear_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-clear.opblock-control__btn")))
        clear_button.click()

        WebDriverWait(driver, 10).until_not(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-clear.opblock-control__btn")))


    except Exception as e:
        print(f'Wystąpił błąd: {str(e)}')
        failed_airports.append(error_entry)


try:
    AUTH_TOKEN = os.getenv("AUTH_TOKEN")
    if not AUTH_TOKEN:
        raise ValueError("Brak tokenu autoryzacyjnego")
    driver.get(url)

    # autoryzacja
    authorize_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.authorize.unlocked span")))
    authorize_button.click()

    # wprowadzenie tokenu w okno
    token_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".auth-container input[type=text]")))
    token_input.send_keys(AUTH_TOKEN)
    print("token wprowadzony")

    # kliknięcie Authorize
    authorizeButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.modal-btn.auth.authorize.button")))
    authorizeButton.click()

    # kliknięcie Close
    close_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.modal-btn.auth.btn-done.button")))
    close_button.click()

    # klik na get airport
    get_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".opblock-summary-method")))
    get_button.click()

    # klik na try-it-out
    try_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.try-out__btn")))
    # przewijanie strony do buttona
    driver.execute_script("arguments[0].scrollIntoView(true);", try_button)
    try_button.click()


    # iteracja po Id
    for airport_id in airports_ids:
        try:
            # wyszukanie pola tekstowego do airportId
            airportId_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".parameters-col_description input[placeholder='airportId']")))
            airportId_input.clear()
            airportId_input.send_keys(str(airport_id))

            # klik execute
            execute_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.execute.opblock-control__btn")))
            execute_button.click()

            #pobranie odpowiedzi (zmiana)
            response_status_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody .response .response-col_status")))
            response_text = response_status_element.text.strip()

            #element odpowiedzi z danymi json lub text
            response_body_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody .response .response-col_description pre")))
            response_body = response_body_element.text.strip()

            # dodawanie do list na podstawie odpowiedzi
            if response_text == "200":
                # airport_data.append({'airport_id': airport_id, 'response_text': response_text})
                response_data = json.loads(response_body) # konwersja stringa na json

                # model danych szablon danych
                airport_entry = {
                    "ORIGIN_AIRPORT_ID": response_data.get("ORIGIN_AIRPORT_ID", airport_id),
                    "DISPLAY_AIRPORT_NAME": response_data.get("DISPLAY_AIRPORT_NAME", "Brak danych"),
                    "ORIGIN_CITY_NAME":response_data.get("ORIGIN_CITY_NAME", "Brak danych") ,
                    "NAME": response_data.get("NAME", "Brak danych")
                }
                if airport_entry not in airport_data:
                    airport_data.append(airport_entry)

            elif response_text == "400":
                error_entry = {
                    "error": f"Airport with id {airport_id} not exist"
                }
                failed_airports.append(error_entry)

            download_and_clear(driver)


        except Exception as e:
            error_entry = {
                "airport_id": airport_id,
                "error": str(e)
            }
            failed_airports.append({error_entry: str(e)})

finally:
    # zapisanie wyników
    with open(os.path.join(output_directory, 'airport_data.json'), 'w', encoding='utf-8') as file:
        json.dump(airport_data, file, ensure_ascii=False, indent=4)
    with open(os.path.join(output_directory, 'failed_airports.json'), 'w', encoding='utf-8') as file:
        json.dump(failed_airports, file, ensure_ascii=False, indent=4)

    # podsumowanie
    print("\n*** Podsumowanie ***")
    print(f"Poprawnie przetworzone: {len(airport_data)} lotniska, niepoprawne: {len(failed_airports)}")

    # zamykanie przeglądarki
    if driver:
        driver.quit()
        print("end processing")

[10874, 11233, 13360, 15008, 11638, 14150, 15323, 14814, 12007, 11337, 13342, 15070, 13244, 12280, 15096, 11641, 13832, 10268, 10397, 15041, 10529, 12119, 11537, 11092, 10581, 13829, 15389, 10140, 12389, 11648, 15023, 11982, 10967, 11525, 10792, 14259, 11637, 10466, 10599, 10208, 15841, 14831, 12898, 13241, 13367, 11481, 14108, 13931, 13873, 10157, 10245, 11146, 13277, 11292, 11109, 13459, 11775, 16218, 14698, 14252, 13256, 13139, 12250, 11259, 11468, 14952, 12402, 14574, 11996, 11977, 11867, 11203, 11995, 15016, 10747, 14905, 12012, 14783, 14730, 10431, 10434, 16869, 10408, 12264, 11618, 15304, 13577, 12954, 11624, 13541, 13422, 14057, 13232, 10800, 14689, 12391, 10868, 14711, 10257, 11067, 10562, 11695, 13796, 14109, 13970, 14193, 11076, 14092, 11122, 11288, 11308, 10754, 12884, 15376, 14588, 11884, 12915, 13851, 14843, 11603, 14457, 12206, 11252, 11905, 15412, 14120, 11980, 14025, 11150, 14709, 15897, 14107, 14006, 14747, 12902, 13061, 12016, 14794, 11921, 10731, 14828, 12889, 12441

 Tutaj zdefiniuj parametry połączenia do API

In [ ]:
# powyżej jest kod

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [ ]:
# powyżej jest kod

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [331]:
# wczytanie danych z pliku json w postaci słowników
with open(os.path.join(output_directory, 'airport_data.json'), 'r', encoding='utf-8') as json_file:
    data = json.load(json_file) # załadunek danych z pliku json
    # print(data)

# konwertowanie danych do ramki metodą from_records
airport_df = pd.DataFrame.from_records(data)

# zapis wyniku do csv   (os.path.join(output_directory,
airport_df.to_csv(os.path.join(output_directory,"airport_data.csv"), index=False, encoding='utf-8')
print("Zapisano plik airport_data.csv")
print(airport_df.shape)

# print(airport_df)
print(airport_df.columns)
# print(airport_df.head())

# spr czy są braki danych NaN
print(airport_df.isnull().sum())

print("\n***   *** ***")
# wyszukanie i zliczenie wierszy "Brak danych" any axis sprawdza czy jest jakikolwiek wpis "Brak danych"
rows_with_missing_data = len(airport_df[(airport_df == "Brak danych").any(axis=1)])
print(rows_with_missing_data)

# usunięcie wierszy "Brak danych" negacja warunku [~...] zachowuje te wiersze które nie mają wpisu "Brak danych"
rows_with_cleaned_data = airport_df[~(airport_df == "Brak danych"). any(axis=1)]
print(rows_with_cleaned_data.shape)


# zapisanie ramki danych po usunięciu braków
rows_with_cleaned_data.to_csv(os.path.join(output_directory, "airport_df_cleaned.csv"), index=False, encoding='utf-8')
print(rows_with_cleaned_data.shape)

Zapisano plik airport_data.csv
(104, 4)
Index(['ORIGIN_AIRPORT_ID', 'DISPLAY_AIRPORT_NAME', 'ORIGIN_CITY_NAME',
       'NAME'],
      dtype='object')
ORIGIN_AIRPORT_ID       0
DISPLAY_AIRPORT_NAME    0
ORIGIN_CITY_NAME        0
NAME                    0
dtype: int64

***   *** ***
7
(97, 4)
(97, 4)


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [339]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == rows_with_cleaned_data.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [ ]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)